# Imports

In [2]:
#Install ipytest package (uncomment only if not already installed)

# !pip install ipytest

In [1]:
#External libraries

import ipytest
import pytest

import numpy as np

import os
import sys

In [2]:
#The module that needs to be tested

#Necessary for relative imports (see https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im)
module_path = os.path.abspath(os.path.join('../../'))           # '../../' is needed because the parent directory is two directories upstream of this test directory
if module_path not in sys.path:
    sys.path.append(module_path)

from kNN_ASMR.kNN_3D import kNN_3D

In [8]:
#Helper functions

from kNN_ASMR.HelperFunctions import create_query_3D

# Setup

In [3]:
ipytest.autoconfig()

# Test functions

In [20]:
help(create_query_3D)

Help on function create_query_3D in module kNN_ASMR.HelperFunctions:

create_query_3D(query_type, query_grid, BoxSize)
    Generates an array of query points; can be either randomly drawn from a uniform distribution defined over the box or put on a uniform grid.
    
    Parameters
    ----------
    query_type : {'grid', 'random'}, str
        the type of query points to be generated; should be 'grid' for query points defined on a uniform grid and 'random' for query points drawn from a uniform random distribution.
    query_grid : int
        the 1D size of the query points array; the total number of query points generated will be ``query_grid**3``.
    BoxSize : float
        the size of the 3D box of the input density field, in Mpc/h.
    
    Returns
    -------
    query_pos : numpy float array of shape ``(query_grid**3, 3)``
        array of query point positions. For each query point in the array, the first, second and third entries are the x, y and z coordinates respectively, i

In [4]:
help(kNN_3D)

Help on module kNN_ASMR.kNN_3D.kNN_3D in kNN_ASMR.kNN_3D:

NAME
    kNN_ASMR.kNN_3D.kNN_3D

FUNCTIONS
    TracerAuto3D(boxsize, kList, BinsRad, QueryPos, TracerPos, ReturnNNdist=False, Verbose=False)
        Computes the $k$NN-CDFs in 3D coordinates (Banerjee & Abel (2021)[^1]) of the provided discrete tracer set (`TracerPos`), 
        evaluated at the provided radial distance scales `BinsRad`, for all $k$ in `kList`. Each $k$NN-CDF measures the probability
        $P_{\geq k}(r)$ of finding at least $k$ tracers in a randomly placed sphere of radius $r$. The $k$NN-CDFs quantify the spatial 
        clustering of the tracers.
                    
        Parameters
        ----------
        kList : list of ints
            the list of nearest neighbours to calculate the distances to. For example, if ``kList = [1, 2, 4]``, the first, second and 
            fourth-nearest neighbour distributions will be computed.
        BinsRad : list of numpy float array
            list of radial di

In [33]:
# #Load data
# kList = [1]
# BinsRad = [np.linspace(10, 45, 3)]
# QueryPos = create_query_3D('grid', 10, 1000.)
# TracerPos = np.load('/home/kaustubh/Projects/kNN-Samrajya/Datasets/illustris_data/HaloPos_4kMostMassive.npy')

# output = kNN_3D.TracerAuto3D(1000., kList, BinsRad, QueryPos, TracerPos)

In [34]:
%%ipytest -vv
#For multiple input-output test pairs

@pytest.mark.parametrize("boxsize", 
                         [-1, np.inf, np.nan, 0, 10, 0.1, 1000.])

def test_TracerAuto3D_boxsize(boxsize):

    #Load data
    kList = [1]
    BinsRad = [np.linspace(10, 45, 3)]
    QueryPos = create_query_3D('grid', 10, 1000.)
    TracerPos = np.load('/home/kaustubh/Projects/kNN-Samrajya/Datasets/illustris_data/HaloPos_4kMostMassive.npy')
    
    #Do the test
    #Check if the function raises a ValueError if the parameters provided are outside the expected range
    with pytest.raises(ValueError):
        output = kNN_3D.TracerAuto3D(boxsize, kList, BinsRad, QueryPos, TracerPos)

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.3.1, pluggy-1.0.0 -- /usr/bin/python3
cachedir: .pytest_cache
hypothesis profile 'default' -> database=DirectoryBasedExampleDatabase(PosixPath('/home/kaustubh/Projects/kNN-Samrajya/Tests/TracerField_3D/.hypothesis/examples'))
rootdir: /home/kaustubh/Projects/kNN-Samrajya/Tests/TracerField_3D
plugins: ligo.skymap-1.0.7, hypothesis-6.131.0, anyio-3.6.2
collecting ... collected 7 items

t_86f9daa56aff420795fa39031c64eb07.py::test_TracerAuto3D_boxsize[-1] PASSED                  [ 14%]
t_86f9daa56aff420795fa39031c64eb07.py::test_TracerAuto3D_boxsize[inf] PASSED                 [ 28%]
t_86f9daa56aff420795fa39031c64eb07.py::test_TracerAuto3D_boxsize[nan] PASSED                 [ 42%]
t_86f9daa56aff420795fa39031c64eb07.py::test_TracerAuto3D_boxsize[0] PASSED                   [ 57%]
t_86f9daa56aff420795fa39031c64eb07.py::test_TracerAuto3D_boxsize[10] 

In [6]:
# %%ipytest

# def test_top_hat_smoothing_2DA_ValueError():

#     #Check if the function raises a ValueError if the parameters provided are outside the expected range
#     with pytest.raises(ValueError):
#         output = hf.top_hat_smoothing_2DA(np.ones(12*(2**6)**2), 3*np.pi)

In [7]:
# %%ipytest

# def test_top_hat_smoothing_2DA():

#     #Check if the smoothed version of a constant field is close to itself within a certain tolerance (in principle they should be exactly equal)
#     #Note that the NSIDE needs to be high enough for this to pass, or you need to reduce the tolerance in np.isclose())
#     NSIDE = 256
#     size = 12*NSIDE**2
#     skymap = np.ones(size)
#     skymap_normalized = skymap/np.sum(skymap)
#     scale = np.deg2rad(0.1)
#     smooth_skymap = hf.top_hat_smoothing_2DA(skymap_normalized, scale)

#     assert np.all(np.isclose(smooth_skymap, skymap_normalized))

#     #Check if the smoothed version of a 0 field is exactly 0
#     NSIDE = 64
#     size = 12*NSIDE**2
#     skymap = np.zeros(size)
#     scale = np.deg2rad(5)
#     smooth_skymap = hf.top_hat_smoothing_2DA(skymap, scale)

#     assert np.all(smooth_skymap==0.0)